# Chat server

## Getting servers into the system

Servers get in via jetstream

In [ ]:
//| export

import { z } from "zod";
import { Record as Server } from "tinychat/api/types/chat/tinychat/core/server.ts";
import { Record as Membership } from "tinychat/api/types/chat/tinychat/core/membership.ts";
import { atURIFromParts } from "tinychat/utils.ts";

const newServerRecordSchema = z.object({
  did: z.string(),
  time_us: z.number(),
  commit: z.object({
    rev: z.string(),
    operation: z.string(),
    collection: z.string(),
    rkey: z.string(),
    cid: z.string().optional(),
    record: z.custom<Server>(),
  }),
}).transform((d) => {
  if (!d.commit) {
    throw new Error(`Invalid record: ${JSON.stringify(d, null, 2)}`);
  }
  return Object.assign({}, d, {
    uri: atURIFromParts({
      did: d.did,
      collection: d.commit.collection,
      rkey: d.commit.rkey,
    }),
  });
});

type NewServerRecord = z.infer<typeof newServerRecordSchema>;

const newMembershipRecordSchema = z
  .object({
    did: z.string(),
    time_us: z.number(),
    commit: z.object({
      rev: z.string(),
      operation: z.string(),
      collection: z.string(),
      rkey: z.string(),
      cid: z.string().optional(),
      record: z.custom<Membership>(),
    }),
  })
  .transform((d) => {
    if (!d.commit) {
      throw new Error(`Invalid record: ${JSON.stringify(d, null, 2)}`);
    }
    return Object.assign({}, d, {
      uri: atURIFromParts({
        did: d.did,
        collection: d.commit.collection,
        rkey: d.commit.rkey,
      }),
    });
  });

type NewMembershipRecord = z.infer<typeof newMembershipRecordSchema>;

const deleteServerRecordSchema = z
  .object({
    did: z.string(),
    time_us: z.number(),
    commit: z.object({
      rev: z.string(),
      operation: z.string(),
      collection: z.string(),
      rkey: z.string(),
      cid: z.string().optional(),
    }),
  })
  .transform((d) => {
    if (!d.commit) {
      throw new Error(`Invalid record: ${JSON.stringify(d, null, 2)}`);
    }
    return Object.assign({}, d, {
      uri: atURIFromParts({
        did: d.did,
        collection: d.commit.collection,
        rkey: d.commit.rkey,
      }),
    });
  });

const deleteMembershipRecordSchema = z
  .object({
    did: z.string(),
    time_us: z.number(),
    commit: z.object({
      rev: z.string(),
      operation: z.string(),
      collection: z.string(),
      rkey: z.string(),
      cid: z.string().optional(),
    }),
  })
  .transform((d) => {
    if (!d.commit) {
      throw new Error(`Invalid record: ${JSON.stringify(d, null, 2)}`);
    }
    return Object.assign({}, d, {
      uri: atURIFromParts({
        did: d.did,
        collection: d.commit.collection,
        rkey: d.commit.rkey,
      }),
    });
  });

type DeleteServerRecord = z.infer<typeof deleteServerRecordSchema>;
type DeleteMembershipRecord = z.infer<typeof deleteMembershipRecordSchema>;

In [ ]:
//| export

import { Jetstream } from "@skyware/jetstream";
import type { Database } from "tinychat/db.ts";
import { TinychatAgent } from "tinychat/agent.ts";
import { ids } from "tinychat/api/lexicons.ts";
import { TID } from "@atproto/common";
import { fetchView, seedMessages, waitForSync } from "tinychat/db.ts";
import {
  ServerSummaryView,
  ServerView,
  validateServerSummaryView,
  validateServerView,
} from "tinychat/api/types/chat/tinychat/server/defs.ts";
import { syncUser } from "tinychat/core/users.ts";

export class Servers {
  constructor(protected db: Database) {}

  public connectToJetstream(jetstream: Jetstream) {
    // keep an eye on servers
    jetstream.on(ids.ChatTinychatCoreServer, async (event) => {
      // we only do creates for now
      if (event.commit.operation === "create") {
        const data = newServerRecordSchema.parse(event);
        await syncUser({ did: event.did, db: this.db });
        this.syncServer(data);
      } else if (event.commit.operation === "delete") {
        this.deleteServer(deleteServerRecordSchema.parse(event));
      }
    });
    // and server memberships
    jetstream.on(ids.ChatTinychatCoreMembership, async (event) => {
      // we only do creates for now
      if (event.commit.operation === "create") {
        const data = newMembershipRecordSchema.parse(event);
        await syncUser({ did: event.did, db: this.db });
        await this.syncMembership(data);
      } else if (event.commit.operation === "delete") {
        this.deleteMembership(deleteMembershipRecordSchema.parse(event));
      }
    });
  }

  async createServer({
    name,
    tc,
  }: {
    name: string;
    tc: TinychatAgent;
  }): Promise<ServerView> {
    const { uri } = await waitForSync<{ uri: string }>({
      db: this.db,
      op: () => {
        return tc.chat.tinychat.core.server.create(
          {
            repo: tc.agent.assertDid,
          },
          {
            name,
            channels: [
              { name: "general", id: TID.nextStr() },
              { name: "random", id: TID.nextStr() },
              { name: "meta", id: TID.nextStr() },
            ],
          },
        );
      },
      sql: ({ uri }) => `SELECT uri FROM servers WHERE uri = '${uri}'`,
    });

    await this.joinServer({ server: uri, tc });

    return (await this.getServers({ uris: [uri] }))[0];
  }

  async joinServer({
    server,
    tc,
  }: {
    server: string;
    tc: TinychatAgent;
  }): Promise<{ uri: string }> {
    return await waitForSync<{ uri: string }>({
      db: this.db,
      op: () => {
        return tc.chat.tinychat.core.membership.create(
          { repo: tc.agent.assertDid },
          {
            server,
            createdAt: new Date().toISOString(),
          },
        );
      },
      sql: ({ uri }) =>
        `SELECT uri FROM server_memberships WHERE uri = '${uri}'`,
    });
  }

  syncServer(server: NewServerRecord) {
    const createChannel = this.db.prepare(
      `INSERT INTO channels (id, name, server) VALUES (:id, :name, :server) ON CONFLICT(id, server) DO NOTHING`,
    );

    this.db.transaction(() => {
      this.db
        .prepare(
          `
        INSERT INTO servers (uri, name, creator) VALUES (
          :uri, :name, :creator
        )`,
        )
        .run({
          uri: server.uri,
          name: server.commit.record.name,
          creator: server.did,
        });

      for (const channel of server.commit.record.channels) {
        createChannel.run({
          id: channel.id,
          name: channel.name,
          server: server.uri,
        });
      }
    })();
  }

 public async syncMembership(m: NewMembershipRecord) {
    this.db
      .prepare(
        `INSERT INTO server_memberships (user, server, uri) VALUES (
        :user, :server, :uri
      )`,
      )
      .run({
        uri: m.uri,
        user: m.did,
        server: m.commit.record.server,
      });

    if (Deno.env.get("SEED_MESSAGES_AFTER_SERVER_CREATION")) {
      console.log(
        "SEED_MESSAGES_AFTER_SERVER_CREATION is set, checking if we need to seed messages",
      );
      const messageCount = this.db
        .prepare(
          `SELECT COUNT(*) FROM messages WHERE server = '${m.commit.record.server}'`,
        )
        .value()![0];

      console.log(
        "Message count: ",
        messageCount,
        "for server: ",
        m.commit.record.server,
      );

      if (messageCount === 0) {
        console.log("Seeding messages for server: ", m.commit.record.server);
        await seedMessages({ db: this.db, server: m.commit.record.server });
      }
    } else {
      console.log(
        "SEED_MESSAGES_AFTER_SERVER_CREATION is NOT set; not seeding messages",
      );
    }
  }

  deleteServer(server: DeleteServerRecord) {
    this.db.prepare(`DELETE FROM servers WHERE uri = '${server.uri}'`).run();
  }

  deleteMembership(membership: DeleteMembershipRecord) {
    this.db
      .prepare(
        `DELETE FROM server_memberships WHERE user = '${membership.did}' AND server = '${membership.uri}'`,
      )
      .run();
  }

  public getServers({
    uris,
    did,
  }: // viewer,
    {
      uris?: string[] | undefined;
      did?: string | undefined;
      viewer?: string | undefined;
    }): ServerView[] {
    let baseWhere = "";
    if (uris && uris.length > 0) {
      baseWhere = `uri IN (${uris.map((u) => `'${u}'`).join(", ")})`;
    } else if (did) {
      baseWhere = `creator__did = '${did}'`;
    }

    // viewer ? `viewer = '${viewer}'` : "",
    const where = [baseWhere]
      .filter((q) => q)
      .join(" AND ")
      .trim();

    return fetchView<ServerView>({
      db: this.db,
      sql: `SELECT * FROM  server_view ${where ? `WHERE ${where}` : ""}`,
      validate: validateServerView,
    });
  }

  public getServersForMember({ did }: { did: string }): ServerView[] {
    return fetchView<ServerView>({
      db: this.db,
      sql: `SELECT * FROM server_with_members_view WHERE member = '${did}'`,
      validate: validateServerView,
    });
  }

  public findServers({
    query,
  }: {
    query?: string | undefined;
  }): ServerSummaryView[] {
    console.log("Finding servers with query: ", query);
    return fetchView<ServerSummaryView>({
      db: this.db,
      sql: "SELECT * FROM  server_view",
      validate: validateServerSummaryView,
    });
  }
}

## Tests

### Test ingestion

In [ ]:
import { startJetstream } from "tinychat/firehose.ts";
import { getDatabase } from "tinychat/db.ts";
import { assertEquals } from "asserts";

Deno.test("server data ingestion", async () => {
  const serverName = "Test server";
  const agent = await TinychatAgent.create();
  const db = getDatabase({ reset: true });
  const servers = new Servers(db);
  const { jetstream, cleanup } = await startJetstream({ db });

  servers.connectToJetstream(jetstream);

  // make sure we do not have any servers yet
  assertEquals(
    db.prepare("SELECT * FROM servers").all().length,
    0,
    "no servers in the system yet",
  );

  await servers.createServer({ name: serverName, tc: agent });

  // make sure we have a server now
  assertEquals(
    db.prepare("SELECT * FROM servers WHERE name = :name").all({
      name: serverName,
    }).length,
    1,
    "server created",
  );

  await cleanup();
});

In [ ]:
import { assertEquals } from "asserts";
import { getDatabase } from "tinychat/db.ts";
import { createDefaultTestUser, createTestUser } from "tinychat/core/users.ts";

const s = {
  did: "did:plc:ubdeopbbkbgedccgbum7dhsh",
  time_us: 1738230288999577,
  commit: {
    rev: "3lgx75ibpx22b",
    operation: "create",
    collection: "chat.tinychat.core.server",
    rkey: "3lgx75ib5fc2b",
    cid: "bafyreigoeopnd7knsghm4xdxfenvf4kwfxoio6yxjp6io7nt6a4wly3fvy",
    record: {
      $type: "chat.tinychat.core.server",
      name: "test-3lgx75hnhht2l",
      channels: [{ id: "3lgx75hnocm2l", name: "general" }],
    },
  },
  uri:
    "at://did:plc:ubdeopbbkbgedccgbum7dhsh/chat.tinychat.core.server/3lgx75ib5fc2b",
};

Deno.test("getServers", () => {
  const db = getDatabase({ reset: true });
  const viewer = "did:plc:uowoeopbbkyuyewdccgbum7dhsh";
  createTestUser({
    db,
    user: {
      did: viewer,
      handle: "bob.com",
    },
  });
  const servers = new Servers(db);

  createDefaultTestUser({ db });
  servers.syncServer(s);

  assertEquals(
    servers.getServers({}).length,
    1,
    "got a server using empty query",
  );
  assertEquals(
    servers.getServers({ did: s.did }).length,
    1,
    "got a server using did",
  );
  assertEquals(
    servers.getServers({ uris: [s.uri] }).length,
    1,
    "got a server using uri",
  );

  // with viewer option
  assertEquals(
    servers.getServers({ viewer: s.did }).length,
    1,
    "got a server using empty query with viewer",
  );
  assertEquals(
    servers.getServers({ did: s.did, viewer: s.did }).length,
    1,
    "got a server using did and viewer",
  );
  assertEquals(
    servers.getServers({ uris: [s.uri], viewer: s.did }).length,
    1,
    "got a server using uri and viewer",
  );

  // viewer who is NOT chat server admin
  // assertEquals(
  //   servers.getServers({ viewer }).length,
  //   1,
  //   "got a server using empty query with viewer"
  // );
});

Deno.test("findServers", () => {
  const db = getDatabase({ reset: true });
  const servers = new Servers(db);
  createDefaultTestUser({ db });
  servers.syncServer(s);
  assertEquals(
    servers.findServers({}).length,
    1,
    "found 1 server",
  );
});

In [ ]:
import { assert, assertEquals } from "asserts";
import { TestDatabase } from "tinychat/db.ts";

Deno.test("delete server", async () => {
  const testServers = await TestDatabase.setup();
  const db = testServers.db;
  const servers = new Servers(db);

  // spot check db before delete

  assertEquals(
    db.prepare(`SELECT * FROM servers`).all().length,
    2,
    "starting with 2 servers",
  );
  assertEquals(
    db.prepare(`SELECT * FROM server_memberships`).all().length,
    4,
    "starting with 4 memberships",
  );
  assertEquals(
    db.prepare(`SELECT * FROM channels`).all().length,
    4,
    "starting with 4 channels",
  );
  // make sure we have messages for the server
  assert(
    db.prepare(`SELECT * FROM messages WHERE server = '${TestDatabase.server}'`)
      .all().length > 10,
    "got a bunch of messages",
  );

  const rkey = TestDatabase.server.split("/").pop()!;

  servers.deleteServer({
    did: TestDatabase.user1,
    time_us: 1738269878791634,
    commit: {
      rev: "3lgydzdyarc2b",
      operation: "delete",
      collection: "chat.tinychat.core.server",
      rkey,
    },
    uri: TestDatabase.server,
  });

  // spot check db after delete
  assertEquals(
    db.prepare(`SELECT * FROM servers`).all().length,
    1,
    "1 server left",
  );
  // make sure server is gone
  assertEquals(
    db.prepare(`SELECT * FROM servers WHERE uri = '${TestDatabase.server}'`)
      .all().length,
    0,
    "server is gone",
  );
  // make sure there are no channels left
  assertEquals(
    db.prepare(`SELECT * FROM channels WHERE server = '${TestDatabase.server}'`)
      .all().length,
    0,
    "channels are gone",
  );
  // make sure there are no memberships left
  assertEquals(
    db.prepare(
      `SELECT * FROM server_memberships WHERE server = '${TestDatabase.server}'`,
    ).all().length,
    0,
    "memberships are gone",
  );
  // make sure there are no messages left
  assertEquals(
    db.prepare(`SELECT * FROM messages WHERE server = '${TestDatabase.server}'`)
      .all().length,
    0,
    "messages are gone",
  );
});

In [ ]:
import { TestDatabase } from "tinychat/db.ts";

Deno.test("getServersForMember", async () => {
  const testServers = await TestDatabase.setup();
  const db = testServers.db;
  const servers = new Servers(db);
  const serversForUser1 = servers.getServersForMember({
    did: TestDatabase.user1,
  });

  assertEquals(
    serversForUser1.length,
    2,
    `user1 (${TestDatabase.user1}) is in 2 servers`,
  );
  assert(
    serversForUser1.find((s) => s.uri === TestDatabase.server),
    "user1 is in server 1",
  );
  assert(
    serversForUser1.find((s) => s.uri === TestDatabase.server2),
    "user1 is in server 2",
  );
});